In [1]:
import matplotlib.pyplot as plt
from datetime import timedelta
import AssetAllocation as AA
import numpy as np
import pandas as pd
import yfinance as yf
from scipy import stats

In [2]:
# ASSET PICKER
def pick_assets(data: pd.DataFrame, current_open_positions: pd.DataFrame, fiscal_date: str):
    # Previous Stocks Evaluation
    predictor = 'Yhat'
    previous_stocks = data[(data['fiscalDateEnding'] == fiscal_date) & (data[predictor] == 0)].merge(current_open_positions.copy(), left_on='Stock', right_on='Asset', how = 'inner')
    previous_stocks = previous_stocks['Stock'].values
    # Add Missing Stocks
    new_assets = 5 - len(previous_stocks)
    try:
        add_assets = data[(data['fiscalDateEnding'] == fiscal_date) & (data[predictor] == 0)]
        if new_assets > len(add_assets):
            new_assets = len(add_assets)
        add_assets = add_assets.sample(n = new_assets)['Stock'].values
        assets_list = list(previous_stocks) + list(add_assets) 
    except:
        assets_list = []
    return assets_list

# OMEGA ASSET ALLOCATION
def omegaAA(data: pd.DataFrame, assets: pd.DataFrame, assets_lists: list, fiscal_date: str, mkt_idx: str = '^GSPC'):
    # Omega Optimization
    rf_rate = data.rf.values[0] / 100
    if len(assets_lists) > 0:
        tickers = assets_lists.copy()
        tickers.append(mkt_idx)
        end_date = pd.to_datetime(fiscal_date)
        start_date = end_date + timedelta(days = -365)
        try:
            omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
        except:
            try:
                omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
            except:
                try:
                    omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
                except:
                    omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
        omega = AA.asset_allocation(data_stocks=omega_prices[omega_prices.columns[:-1]], data_benchmark=omega_prices[omega_prices.columns[-1]].to_frame(), rf=rf_rate)
        omega_weights = omega.omega(n_port=1)
    else:
        omega_weights = []
    # RF
    rf_percentage = (5 - len(omega_weights)) * .2
    if rf_percentage > 0:
        assets_lists = assets_lists + ['Rf']
        omega_weights = np.array(omega_weights) * (1 - rf_percentage)
        omega_weights = np.concatenate((omega_weights, [rf_percentage]))
        omega_weights = list(omega_weights).copy()

    # Assets DF
    new_assets = pd.DataFrame([ pd.to_datetime([fiscal_date for i in range(len(omega_weights))]),
                             assets_lists, omega_weights],
                           index=['Date','Stock','W']).transpose()
    assets = pd.concat([assets, new_assets], axis = 0, ignore_index = True)

    # Omega Weights
    omega_weights = {asset:[weight] for asset,weight in zip(assets_lists,omega_weights)}
    omega_weights = pd.DataFrame.from_dict(omega_weights).T.reset_index().rename(columns = {'index':'Stock',0:'Weight'})

    return omega_weights, assets

## Sell RF
def open_positions(current_open_positions: pd.DataFrame, consult_asset: str):
    return consult_asset in  current_open_positions.Asset.values

def valuate_position(current_open_positions: pd.DataFrame, consult_asset: str):
    rf_open_position = current_open_positions[current_open_positions['Asset'] == consult_asset]['X'].values[0]  
    position_date = current_open_positions[current_open_positions['Asset'] == consult_asset]['Date'].values[0]
    return   rf_open_position, position_date

def close_position(current_open_positions: pd.DataFrame):
    current_open_positions = current_open_positions[current_open_positions['Asset'] != 'Rf']
    return current_open_positions

def report_sale(operations: pd.DataFrame, rf_open_position: float, position_date: str, fiscal_date: str):
    previous_rf = operations[(operations['Date'] == position_date) & (operations['Asset'] == 'Rf') & (operations['Type'] == 'Buy')]['Price'].values[0] / 100
    income_risk_free =  rf_open_position * ( 1 + previous_rf *  3/12)
    sell_operation = pd.DataFrame([fiscal_date,'Rf',-income_risk_free,previous_rf,-income_risk_free,'Sell'], index = ['Date','Asset','X','Price','Position','Type']).T
    operations = pd.concat([operations,sell_operation], axis = 0, ignore_index = True)
    return operations, income_risk_free

def update_funds(income_risk_free: float):
    cash = income_risk_free
    return cash
    
def sell_rf(current_open_positions: pd.DataFrame, operations: pd.DataFrame, fiscal_date: str):
    if open_positions(current_open_positions, 'Rf'):
        rf_open_position, position_date = valuate_position(current_open_positions, 'Rf')
        current_open_positions = close_position(current_open_positions)
        operations, income_risk_free = report_sale(operations, rf_open_position, position_date, fiscal_date)
        cash = update_funds(income_risk_free)
    else:
        income_risk_free = 0
        cash = update_funds(income_risk_free)
    return current_open_positions, operations, cash

## Current Port Value
def previous_capitals_open(current_open_positions: pd.DataFrame):
    return len(current_open_positions[current_open_positions['Asset'] != 'Rf']['Asset'].values) > 0

def get_market_prices(tickers, start_date: str, end_date: str):
    # Error Handling with API Connection
    try:
        prices_new = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
    except:
        try:
            prices_new = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
        except:
            try:
                prices_new = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
            except:
                prices_new = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
    if len(tickers) > 1:
        prices_new = prices_new.iloc[0]
        prices_new = prices_new.to_frame()
        prices_new.columns = ['Price']
    else:
        prices_new = pd.DataFrame(prices_new.iloc[0], index = [tickers[0]], columns= ['Price'])
    return prices_new

def current_position_value(current_open_positions: pd.DataFrame, fiscal_date: str):
    tickers_previous = list(current_open_positions[current_open_positions['Asset'] != 'Rf']['Asset'].values)

    end_date = pd.to_datetime(fiscal_date) + timedelta(days=7)
    end_date = end_date.strftime('%Y-%m-%d')

    previous_portfolio = current_open_positions[['Asset','X']].set_index('Asset')
    prices_new = get_market_prices(tickers_previous, start_date=fiscal_date, end_date= end_date)
    previous_portfolio = previous_portfolio.merge(prices_new, left_index = True, right_index = True, how = 'left')
    previous_portfolio['Pos'] = previous_portfolio['X'] * previous_portfolio['Price']
    previous_portfolio = previous_portfolio.rename(columns = {'X':'X_1'})
    capitals_value = previous_portfolio['Pos'].sum()
    return previous_portfolio, capitals_value

def overall_portfolio_value(capitals_value: float, cash: float):
    return capitals_value + cash

def no_previous_capital_open():
    return pd.DataFrame(columns = ['Stock','X_1']).set_index('Stock')

def current_portfolio_value(current_open_positions: pd.DataFrame, cash: float, fiscal_date: str):
    if previous_capitals_open(current_open_positions):
        previous_portfolio, capitals_value = current_position_value(current_open_positions, fiscal_date)
        cash = overall_portfolio_value(capitals_value, cash)
    else:
        previous_portfolio = no_previous_capital_open()
        cash = cash # No aditional cash from capitals

    return previous_portfolio, cash

## Trade Capitals
def adjustments_to_portofolio(omega_weights: pd.DataFrame, previous_portfolio: pd.DataFrame, cash: float, fiscal_date: str, comission: float = .00125 * 1.16):
    Xt = omega_weights[omega_weights['Stock'] != 'Rf'].copy()
    end_date = pd.to_datetime(fiscal_date) + timedelta(days=7)
    end_date = end_date.strftime('%Y-%m-%d')
    tickers = list(Xt['Stock'].values) + list(previous_portfolio.index.values)

    if len(tickers) > 0:
        try:
            prices_new = get_market_prices(tickers=tickers, start_date=fiscal_date, end_date= end_date)
        except:
            prices_new = pd.DataFrame(columns = ['Price'])
    else:
        prices_new = pd.DataFrame(columns = ['Price'])

    Xt = Xt.rename(columns={'Stock':'Asset'})
    Xt = Xt.set_index('Asset')
    X = Xt.join(previous_portfolio['X_1'], how='outer').fillna(0.0)
    X = X.merge(prices_new, left_index = True, right_index = True, how = 'left') 
    X["X"] = (X['Weight'] * cash / X['Price']).apply(lambda x: np.floor(x))
    X['Trade'] = X['X'] - X['X_1']
    
    # Only capitals
    X = X.reset_index().rename(columns = {'Stock':'Asset','index':'Asset'})

    # Cash withdrawl
    stocks_cash = cash
    cash = cash * ( 1 - X['Weight'].sum() )

    if (X['Weight'] * stocks_cash).sum() > (X['X'] * X['Price']).sum():
        remaining = (X['Weight'] * stocks_cash).sum() - (X['X'] * X['Price']).sum()
        cash += remaining
    # Sell 
    to_sell = X['Trade']<0
    if len(X[to_sell]) > 0:
        assets_to_sell = np.floor((X[to_sell].Trade) / (1 + comission))
        cash_gain_per_asset = assets_to_sell * X[to_sell]['Price'] 
        operations_sell = assets_to_sell.to_frame().rename(columns = {'Trade':'X'})
        operations_sell['Price'] = X[to_sell]['Price'].values
        operations_sell['Asset'] = X[to_sell]['Asset'].values
        operations_sell['Position'] = cash_gain_per_asset.T.values
        operations_sell['Type'] = 'Sell'
        operations_sell['Date'] = fiscal_date
    else:
        #net_cash_gain = 0
        operations_sell = pd.DataFrame(columns=['Date','Asset','X','Price','Position','Type'])
    
    # Buy
    to_buy = X['Trade']>0
    if len(X[to_buy]) > 0:
        assets_to_buy = np.floor((X[to_buy].Trade) / (1 + comission))
        cash_invest_per_asset = assets_to_buy * X[to_buy]['Price']
        #net_cash_invest = cash_invest_per_asset.sum(axis=1)
        operations_buy = assets_to_buy.to_frame().rename(columns = {'Trade':'X'})
        operations_buy['Price'] = X[to_buy]['Price'].values
        operations_buy['Asset'] = X[to_buy]['Asset'].values
        operations_buy['Position'] = cash_invest_per_asset.T.values
        operations_buy['Type'] = 'Buy'
        operations_buy['Date'] = fiscal_date
    else:
        operations_buy = pd.DataFrame(columns=['Date','Asset','X','Price','Position','Type'])

    new_operations = pd.concat([operations_sell,operations_buy], axis = 0, ignore_index = True)
    return new_operations, cash

def close_capitals_position(current_open_positions: pd.DataFrame, new_operations: pd.DataFrame):
    sell_operations = new_operations[new_operations['Type'] == 'Sell']
    amount_open = current_open_positions[['Asset','X','Price']]
    amount_to_close = sell_operations[['Date','Asset','X','Price']]
    for stock in amount_to_close.Asset.values:
        if stock in amount_open.Asset.values:
            asset_open = amount_open[amount_open.Asset == stock]
            asset_to_close = amount_to_close[amount_to_close.Asset == stock]
            remaining_open = asset_open.X.iloc[0] + asset_to_close.X.iloc[0]
            if remaining_open > 0:
                if remaining_open < 10:
                    current_open_positions = current_open_positions[current_open_positions['Asset'] != stock]
                else:
                    price = asset_open.Price.iloc[0]
                    new_open_date = asset_to_close.Date.iloc[0]
                    current_open_positions = current_open_positions[current_open_positions['Asset'] != stock]
                    new_entry = pd.DataFrame([new_open_date, stock, remaining_open, price], index=['Date','Asset','X','Price']).T
                    current_open_positions = pd.concat([current_open_positions, new_entry], axis=0, ignore_index=True)
            else:
                current_open_positions = current_open_positions[current_open_positions['Asset'] != stock]
        else:
            pass
    return current_open_positions

def open_capitals_position(current_open_positions: pd.DataFrame, new_operations: pd.DataFrame): 
    buy_operations = new_operations[new_operations['Type'] == 'Buy']
    amount_open = current_open_positions[['Asset', 'X', 'Price']]
    amount_to_open = buy_operations[['Date', 'Asset', 'X', 'Price']]
    for stock in amount_to_open['Asset'].values:
        if stock in amount_open['Asset'].values:
            asset_open = amount_open[amount_open['Asset'] == stock]
            asset_to_open = amount_to_open[amount_to_open['Asset'] == stock]
            remaining_open = asset_open['X'].iloc[0] + asset_to_open['X'].iloc[0]
            price = (asset_open['Price'].iloc[0] * asset_open['X'].iloc[0] + asset_to_open['Price'].iloc[0] * asset_to_open['X'].iloc[0]) / remaining_open 
            current_open_positions = current_open_positions[current_open_positions['Asset'] != stock]
            new_open_date = asset_to_open['Date'].iloc[0]
            new_entry = pd.DataFrame([[new_open_date, stock, remaining_open, price]], columns=['Date', 'Asset', 'X', 'Price'])
            current_open_positions = pd.concat([current_open_positions, new_entry], ignore_index=True)
        else:
            asset_to_open = amount_to_open[amount_to_open['Asset'] == stock]
            remaining_open = asset_to_open['X'].iloc[0]
            price = asset_to_open['Price'].iloc[0]
            new_open_date = asset_to_open['Date'].iloc[0]
            new_entry = pd.DataFrame([[new_open_date, stock, remaining_open, price]], columns=['Date', 'Asset', 'X', 'Price'])
            current_open_positions = pd.concat([current_open_positions, new_entry], ignore_index=True)
    return current_open_positions

def report_operations(operations: pd.DataFrame, new_operations: pd.DataFrame):
    return pd.concat([operations, new_operations], axis=0, ignore_index=True)

def trade_capitals(current_open_positions: pd.DataFrame, operations: pd.DataFrame, omega_weights:pd.DataFrame, 
                   previous_portfolio: pd.DataFrame, cash: float, fiscal_date: pd.DataFrame, comission: float = .00125 * 1.16):
    
    new_operations, cash = adjustments_to_portofolio(omega_weights, previous_portfolio, cash, fiscal_date, comission)
    current_open_positions = close_capitals_position(current_open_positions, new_operations)
    current_open_positions = open_capitals_position(current_open_positions, new_operations)
    operations = report_operations(operations, new_operations)
    return current_open_positions, operations, cash
        
## Buy RF
def invest_in_rf(cash: float):
    return cash > 0

def open_rf_position(data: pd.DataFrame, current_open_positions: pd.DataFrame, cash: float, fiscal_date: str):
    rf = data[data['fiscalDateEnding'] == fiscal_date]['rf'].iloc[0] / 100
    buy_risk_free = cash
    open_pos = pd.DataFrame([fiscal_date,'Rf',buy_risk_free,rf], index = ['Date','Asset','X','Price']).T
    current_open_positions = pd.concat([current_open_positions,open_pos], axis = 0, ignore_index=True)
    return current_open_positions

def report_operation_rf(data: pd.DataFrame, operations: pd.DataFrame, cash: float, fiscal_date: pd.DataFrame):
    rf = data[data['fiscalDateEnding'] == fiscal_date]['rf'].iloc[0] / 100
    buy_risk_free = cash
    new_op = pd.DataFrame([fiscal_date,'Rf',buy_risk_free,rf,buy_risk_free,'Buy'], index = ['Date','Asset','X','Price','Position','Type']).T
    operations = pd.concat([operations,new_op], axis = 0, ignore_index=True)
    return operations

def buy_rf(data: pd.DataFrame, current_open_positions: pd.DataFrame, operations: pd.DataFrame, cash:float, fiscal_date: str):
    if invest_in_rf:
        current_open_positions = open_rf_position(data, current_open_positions, cash, fiscal_date)
        operations = report_operation_rf(data, operations, cash, fiscal_date)
    else:
        pass
    return current_open_positions, operations

## Reporting
def exercise_report(current_open_positions: pd.DataFrame):

    stocks_value = current_open_positions[current_open_positions['Asset'] != 'Rf']
    if len(stocks_value) > 0:
        stocks_value = stocks_value['X'] * stocks_value['Price']
        stocks_value = stocks_value.sum()
    else:
        stocks_value = 0

    rf_value = current_open_positions[current_open_positions['Asset'] == 'Rf']
    if len(rf_value) > 0:
        rf_value = rf_value['X'].iloc[0]
    else:
        rf_value = 0
    
    return stocks_value + rf_value

## Trade Function
def TradeCapitals(data, current_open_positions: pd.DataFrame, operations: pd.DataFrame, omega_weights: pd.DataFrame, fiscal_date: str,initial_cycle: bool = False ,initial_capital: int = 1000000, comission: float = .00025):
    current_open_positions, operations, cash = sell_rf(current_open_positions, operations, fiscal_date)
    if initial_cycle:
        cash = initial_capital
    previous_portfolio, cash = current_portfolio_value(current_open_positions, cash, fiscal_date)
    current_open_positions, operations, cash = trade_capitals(current_open_positions, operations, omega_weights, previous_portfolio, cash, fiscal_date, comission)
    current_open_positions, operations = buy_rf(data, current_open_positions, operations, cash, fiscal_date)
    return current_open_positions, operations

## Backtesting Function
def BackTesting(data: pd.DataFrame):
    assets = pd.DataFrame(columns=['Date','Stock','W'])
    current_open_positions = pd.DataFrame(columns=['Date','Asset','X','Price'])
    operations = pd.DataFrame(columns=['Date','Asset','X','Price','Position','Type'])
    first_date = pd.to_datetime(data['fiscalDateEnding'].unique()[0]).strftime('%Y-%m-%d')
    wo_abc = data[data['Stock'] != 'ABC']
    value_in_time = []

    for fiscal_date in data['fiscalDateEnding'].unique():
        fiscal_date = pd.to_datetime(fiscal_date).strftime('%Y-%m-%d')
        assets_list = pick_assets(data=wo_abc, current_open_positions=current_open_positions, fiscal_date=fiscal_date)
        omega_weights, assets = omegaAA(data=wo_abc, assets=assets, assets_lists=assets_list, fiscal_date=fiscal_date)
        if fiscal_date == first_date:
            current_open_positions, operations = TradeCapitals(data, current_open_positions, operations, omega_weights,
                                                                        fiscal_date, True)
        else:
            current_open_positions, operations = TradeCapitals(data, current_open_positions, operations, omega_weights, fiscal_date)
        value_in_fd = exercise_report(current_open_positions)
        value_in_time.append(value_in_fd)
    value_in_time = pd.DataFrame(value_in_time, index=data['fiscalDateEnding'].unique(), columns=['Portfolio Value'])

    return value_in_time, operations

def return_next_q(sim_operations):
    last_port = sim_operations[(sim_operations.Date == sim_operations.Date.max()) & (sim_operations.Type == 'Buy')]
    capitals = sim_operations[sim_operations.Asset != 'Rf']
    capitals = list(last_port.Asset.values)
    prices = get_market_prices(capitals, start_date = '2023-09-30', end_date = '2023-10-04')
    prices = prices.reset_index()
    prices = prices.rename(columns = {'index':'Asset', 'Price':'New_Price'})
    last_port = last_port.merge(prices, how = 'left', left_on = 'Asset', right_on = 'Asset')
    last_port['New_Position'] = last_port['X'] * last_port['New_Price']
    previous_position = last_port['Position'].sum()
    new_position = last_port['New_Position'].sum()
    new_position += last_port[last_port.Asset == 'Rf'].Position.values[0]
    return new_position / previous_position -1

In [3]:
data = pd.read_excel("Data/data.xlsx").drop("Unnamed: 0", axis=1)

In [4]:
#tickers to keep
valid_tickers = yf.download(data.Stock.unique().tolist(), start="2018-12-31", end="2023-06-30", progress=False)['Adj Close']
tickers_to_keep = valid_tickers.dropna(axis=1).columns.values.tolist()
data = data[data['Stock'].isin(tickers_to_keep)]


1 Failed download:
['ABC']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [5]:
valid_tickers.dropna(axis=1)

A        AAL         AAP        AAPL        ABBV  \
Date                                                                    
2018-12-31   65.199593  31.599045  145.215164   37.850136   72.990067   
2019-01-02   63.488911  31.963156  145.639389   37.893337   70.646545   
2019-01-03   61.149986  29.581665  150.185974   34.118881   68.318832   
2019-01-04   63.266628  31.530161  146.460175   35.575390   70.519859   
2019-01-07   64.610039  32.425678  148.608963   35.496212   71.549118   
...                ...        ...         ...         ...         ...   
2023-06-23  118.952553  16.260000   65.118805  186.182953  133.097458   
2023-06-26  117.906670  16.440001   66.725212  184.776733  132.147598   
2023-06-27  116.581871  17.350000   67.746567  187.559280  129.846405   
2023-06-28  115.914497  17.549999   67.508583  188.746124  129.758270   
2023-06-29  117.587921  17.600000   67.657318  189.085205  130.463303   

                   ABT       ACGL         ACN        ADBE         ADI  ...  \
Date                                                                   ...   
2018-12-31   66.510483  26.719999  131.469055  226.240005   78.424202  ...   
2019-01-02   63.908157  26.190001  131.077438  224.570007   78.506416  ...   
2019-01-03   60.892075  25.780001  126.602242  215.699997   73.764267  ...   
2019-01-04   62.630013  26.389999  131.524963  226.190002   75.555122  ...   
2019-01-07   63.567951  26.330000  131.981812  229.259995   76.030243  ...   
...                ...        ...         ...         ...         ...  ...   
2023-06-23  106.942276  72.410004  296.122650  484.720001  185.060318  ...   
2023-06-26  107.397568  71.220001  295.110565  479.510010  185.458420  ...   
2023-06-27  106.684944  71.139999  298.484253  489.269989  191.469849  ...   
2023-06-28  106.506790  71.680000  299.387238  482.429993  189.001572  ...   
2023-06-29  106.556282  73.029999  305.946106  483.769989  188.583572  ...   

                    RL        RTX       SCHW         STZ          T  \
Date                                                                  
2018-12-31   93.528275  59.220474  38.620461  149.405624  14.432829   
2019-01-02   95.318207  60.232677  38.666958  152.397064  14.938534   
2019-01-03   91.015129  57.551968  37.867203  150.130249  14.958759   
2019-01-04   94.730591  59.520790  39.466705  154.793976  15.343094   
2019-01-07   96.457245  59.493000  39.745678  158.045532  15.621234   
...                ...        ...        ...         ...        ...   
2023-06-23  118.111404  95.534225  52.725212  240.665939  14.894809   
2023-06-26  117.212875  95.179344  52.963207  240.705643  15.068340   
2023-06-27  119.967697  95.159622  54.758064  245.243332  15.270794   
2023-06-28  120.520630  94.508995  55.313377  245.034805  15.212950   
2023-06-29  122.308922  96.411598  56.662003  245.104324  15.405762   

                  TAP       TECH        WRB         XOM       XRAY  
Date                                                                
2018-12-31  50.169975  35.376957  29.943958   52.801029  35.421394  
2019-01-02  51.000782  34.690048  29.340300   53.962517  36.068707  
2019-01-03  51.697586  33.267349  28.854124   53.133987  36.087746  
2019-01-04  53.341328  35.320744  29.299786   55.093014  37.134865  
2019-01-07  53.225197  35.660534  29.384865   55.379509  37.801220  
...               ...        ...        ...         ...        ...  
2023-06-23  65.749847  76.441002  57.664814  100.647057  38.146816  
2023-06-26  65.243164  76.610611  58.110600  102.504700  39.010185  
2023-06-27  65.382256  75.413406  58.179943  102.760262  39.605606  
2023-06-28  65.223297  75.912239  57.575657  103.595703  39.278126  
2023-06-29  65.193489  79.164619  58.566288  104.873444  39.347836  

[1132 rows x 378 columns]

In [6]:
#n_sim = 10
#simulations = [np.array(BackTesting(data=data)[0]) for i in range(n_sim)]

In [ ]:
# with try to skip errors
n_sim = 100
simulations = []

for i in range(n_sim):
    try:
        result = np.array(BackTesting(data=data)[0])
        simulations.append(result)
    except Exception as e:
        print(f"Error in simulation {i + 1}: {e}")

Error in simulation 1: Unable to coerce to DataFrame, shape must be (170, 5): given (249, 1)
Error in simulation 10: Unable to coerce to DataFrame, shape must be (3, 6): given (253, 1)


In [ ]:
avrg_port = np.sum(simulations, axis=0) / len(simulations)
avrg_port_ret = pd.DataFrame(avrg_port).pct_change().dropna()
avrg_port_ret = avrg_port_ret.rename(columns={0: 'Quarter Returns'})

In [ ]:
avrg_port

In [ ]:
benchmark = pd.DataFrame(yf.download("SPY", start="2018-12-31", end="2023-06-30", progress=False)['Adj Close'])
benchmark_ret = benchmark['Adj Close'].resample('Q').ffill().pct_change().dropna().drop("2022-12-31")

In [ ]:
port_ret_series_df = avrg_port_ret
port_ret_series = np.array(port_ret_series_df).reshape(17,)
bench_ret_series = benchmark_ret
initial_capital = 1000000

# Annual performance and volatility metrics
port_return = port_ret_series_df['Quarter Returns'].mean() * 4
port_volatility = port_ret_series_df['Quarter Returns'].std() * np.sqrt(4)
benchmark_return = bench_ret_series.mean() * 4

# Assuming risk-free rate at the start of the strategy
rf = data['rf'][0] / 100

# Sharpe Ratio
sharpe_ratio = ((port_return - rf) / port_volatility)

# Information Ratio
active_returns = port_ret_series - bench_ret_series
information_ratio = (np.mean(active_returns) / np.std(active_returns))

# Beta
beta, _, _, _, _ = stats.linregress(bench_ret_series, port_ret_series)

# Jensen's Alpha
jensens_alpha = (port_return - rf) - (benchmark_return - rf)*beta

# Treynor Ratio
treynor_ratio = ((port_return - rf) / beta) * 100

# Sortino Ratio
downside_returns = port_ret_series_df[port_ret_series_df['Quarter Returns'] < 0]['Quarter Returns']
sortino_ratio = ((port_return - rf) / downside_returns.std())

# Omega Ratio
threshold_return = 0
excess_returns = port_ret_series_df['Quarter Returns'] - rf
omega_ratio = np.sum(np.where(excess_returns > threshold_return, 1, 0)) / np.sum(np.where(excess_returns < threshold_return, 1, 0))

# VaR at 95% and 99%
var_95 = np.percentile(port_ret_series_df['Quarter Returns'], 5) * 100
var_99 = np.percentile(port_ret_series_df['Quarter Returns'], 1) * 100

# Expected Shortfall at 95% and 99%
es_95 = port_ret_series_df[port_ret_series_df['Quarter Returns'] <= var_95 / 100]['Quarter Returns'].mean() * 100
es_99 = port_ret_series_df[port_ret_series_df['Quarter Returns'] <= var_99 / 100]['Quarter Returns'].mean() * 100

# Displaying the results
print("--- Market conditions ---")
print(f"Benchmark Return: {benchmark_return*100:.4f}%")
print(f"Risk Free (FED): {rf*100:.4f}%")

print("")

print("--- Portfolio stats ---")
print(f"Annual Return: {port_return*100:.4f}%")
print(f"Annual Volatility: {port_volatility*100:.4f}%")
print(f"Beta: {beta:.4f}")
print("--- Financial ratios ---")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Information Ratio: {information_ratio:.4f}")
print(f"Jensen's Alpha: {jensens_alpha:.4f}")
print(f"Treynor Ratio: {treynor_ratio:.4f}%")
print(f"Sortino Ratio: {sortino_ratio:.4f}")
print(f"Omega Ratio: {omega_ratio:.4f}")

print("--- Risk measures ---")
print(f"VaR at 95%: {var_95:.4f}% = ${var_95*initial_capital:.2f}")
print(f"VaR at 99%: {var_99:.4f}% = ${var_99*initial_capital:.2f}")
print(f"Expected Shortfall at 95%: {es_95:.4f}% = ${es_95*initial_capital:.2f}")
print(f"Expected Shortfall at 99%: {es_99:.4f}% = ${es_99*initial_capital:.2f}")

In [ ]:
avrg_port = pd.DataFrame(avrg_port, index=benchmark['Adj Close'].resample('Q').ffill().pct_change().drop("2022-12-31").index)


Plots

In [ ]:
dates = data["fiscalDateEnding"].unique()

In [ ]:
avrg_port_1d = avrg_port.flatten()

# Create a DataFrame with 'Dates' as the index
plot_data = pd.DataFrame({'Port': avrg_port_1d}, index=dates)

In [ ]:
plot_data

In [ ]:
benchmark_data = [239.210373, 274.78662109375, 285.6192321777344, 288.01544189453125, 313.1404113769531, 252.854584,
 303.834534, 331.296051, 371.444244, 395.037598, 428.059998, 429.140015, 474.959991, 451.640015,
 377.250000, 377.250000, 409.390015, 438.1099853515625]

In [ ]:
plot_data['benchmark_data'] = benchmark_data

In [ ]:
plot_data

In [ ]:
merged_data = pd.merge(plot_data, benchmark, left_index=True, right_index=True, how='inner')

In [ ]:
plot_data['Normalized Portfolio'] = plot_data['Port'] / plot_data['Port'].iloc[0]
plot_data['Normalized Benchmark'] = plot_data['benchmark_data'] / plot_data['benchmark_data'].iloc[0]
plot_data

In [ ]:
# Assuming 'merged_data' is your DataFrame
# Plot the normalized time series on the same graph
plt.plot(plot_data.index, plot_data['Normalized Portfolio'], label='Portafolio promedio', color='blue')
plt.plot(plot_data.index, plot_data['Normalized Benchmark'], label='Benchmark', color='red')

plt.xlabel('Time')
plt.ylabel('Normalized Values')
plt.legend()
plt.show()

In [ ]:
plt.plot(avrg_port);

In [ ]:
plt.plot(benchmark_toplot);

In [ ]:
plt.boxplot(avrg_port_ret);